In [1]:
import pandas as pd
import numpy as np
import os
import copy
import math

In [2]:
import importlib
import euler_utils
importlib.reload(euler_utils)

<module 'euler_utils' from '/Users/francescmartiescofet/Documents/ETH/IVT/eqasim-java/switzerland/WaitTimeValidation/euler_utils.py'>

<module 'euler_utils' from '/Users/francescmartiescofet/Documents/ETH/IVT/eqasim-java/switzerland/WaitTimeValidation/euler_utils.py'>

In [3]:
main_folder = '/Users/francescmartiescofet/Documents/ETH/IVT/eqasim-java/switzerland/SA_scenario/testDrtZones'

In [4]:
last_iter = 5

In [ ]:
no_rej_4000 = euler_utils.read_output(main_folder + 'no_rejections_4000', last_iter)

In [5]:
no_rej_4000 = euler_utils.read_output(main_folder, last_iter)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 35.20it/s]



In [6]:
def check_global_avg(global_avg, drt_legs, drt_trips_stats):
    for it, (it_global_avg, it_drt_legs, it_drt_trips_stats) in enumerate(zip(global_avg, drt_legs, drt_trips_stats)):
        print('Iteration ' + str(it))
        print('Global avg: ')
        display(it_global_avg)
        p = [0.05,.25,.5,.75,.95]
        print('Waiting time description from legs: ' + str(it_drt_trips_stats.waitTime.describe(percentiles=p)))
        print('Delay factor average from legs: ' + str(it_drt_trips_stats.delayFactor.describe(percentiles=p)))
        print('\n')

In [7]:
check_global_avg(no_rej_4000['global_avg'], no_rej_4000['drt_legs'], no_rej_4000['drt_trips_stats'])

Iteration 0
Global avg: 
Iteration 0
Global avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,delayFactor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Waiting time description from legs: count       0
unique      0
top       NaN
freq      NaN
Name: waitTime, dtype: object
Delay factor average from legs: count       0
unique      0
top       NaN
freq      NaN
Name: delayFactor, dtype: object


Iteration 1
Global avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,delayFactor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Waiting time description from legs: count       0
unique      0
top       NaN
freq      NaN
Name: waitTime, dtype: object
Delay factor average from legs: count       0
unique      0
top       NaN
freq      NaN
Name: delayFactor, dtype: object


Iteration 1
Global avg: 


...

,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,70.636364,61.000000,61.000000,61.000000,61.000000,77.50000,123.900000,126.0
1,delayFactor,1.319256,1.231308,1.040404,1.042916,1.109175,1.35186,2.270435,2.8


Waiting time description from legs: count     33.000000
mean      70.636364
std       18.253113
min       61.000000
5%        61.000000
25%       61.000000
50%       61.000000
75%       75.000000
95%      108.600000
max      126.000000
Name: waitTime, dtype: float64
Delay factor average from legs: count    33.000000
mean      1.319256
std       0.343915
min       1.040404
5%        1.048306
25%       1.111570
50%       1.231308
75%       1.340909
95%       1.867570
max       2.800000
Name: delayFactor, dtype: float64


Iteration 5
Global avg: 
Waiting time description from legs: count     33.000000
mean      70.636364
std       18.253113
min       61.000000
5%        61.000000
25%       61.000000
50%       61.000000
75%       75.000000
95%      108.600000
max      126.000000
Name: waitTime, dtype: float64
Delay factor average from legs: count    33.000000
mean      1.319256
std       0.343915
min       1.040404
5%        1.048306
25%       1.111570
50%       1.231308
75%       1.340909

,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,76.944444,61.000000,61.000000,61.000000,61.000000,89.750000,138.050000,144.000000
1,delayFactor,1.265525,1.203749,1.001562,1.034578,1.072723,1.337578,1.794274,2.043478


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,76.944444,61.000000,61.000000,61.000000,61.000000,89.750000,138.050000,144.000000
1,delayFactor,1.265525,1.203749,1.001562,1.034578,1.072723,1.337578,1.794274,2.043478


Waiting time description from legs: count     36.000000
mean      76.944444
std       25.469247
min       61.000000
5%        61.000000
25%       61.000000
50%       61.000000
75%       89.250000
95%      131.000000
max      144.000000
Name: waitTime, dtype: float64
Delay factor average from legs: count    36.000000
mean      1.265525
std       0.233902
min       1.001562
5%        1.043095
25%       1.077411
50%       1.203749
75%       1.330917
95%       1.681301
max       2.043478
Name: delayFactor, dtype: float64


Waiting time description from legs: count     36.000000
mean      76.944444
std       25.469247
min       61.000000
5%        61.000000
25%       61.000000
50%       61.000000
75%       89.250000
95%      131.000000
max      144.000000
Name: waitTime, dtype: float64
Delay factor average from legs: count    36.000000
mean      1.265525
std       0.233902
min       1.001562
5%        1.043095
25%       1.077411
50%       1.203749
75%       1.330917
95%       1.681301
max  

The interpolation method for the percentile is a little bit different between pandas and java but should be correct once we have more data points.

In [8]:
def check_global_moving_window_avg(global_moving_avg, drt_legs, moving_window, drt_trips_stats):
    for it, it_global_moving_avg in enumerate(global_moving_avg):
        start = 0
        if it >= moving_window:
            start = it - moving_window + 1
        it_drt_legs = pd.concat([drt_legs[j] for j in range(start, it+1)])
        it_drt_trips_stats = pd.concat([drt_trips_stats[j] for j in range(start, it+1)])
        print('Iteration ' + str(it))
        print('Global moving avg: ')
        display(it_global_moving_avg)
        p = [0.05,.25,.5,.75,.95]
        print('Waiting time description from legs: ' + str(it_drt_trips_stats.waitTime.describe(percentiles=p)))
        print('Delay factor average from legs: ' + str(it_drt_trips_stats.delayFactor.describe(percentiles=p)))
        print('\n')
        

In [9]:
check_global_moving_window_avg(no_rej_4000['global_moving_avg'], no_rej_4000['drt_legs'], 5, no_rej_4000['drt_trips_stats'])

Iteration 0
Global moving avg: 
Iteration 0
Global moving avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,delayFactor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,delayFactor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Waiting time description from legs: count       0
unique      0
top       NaN
freq      NaN
Name: waitTime, dtype: object
Delay factor average from legs: count       0
unique      0
top       NaN
freq      NaN
Name: delayFactor, dtype: object


Iteration 1
Global moving avg: 
Waiting time description from legs: count       0
unique      0
top       NaN
freq      NaN
Name: waitTime, dtype: object
Delay factor average from legs: count       0
unique      0
top       NaN
freq      NaN
Name: delayFactor, dtype: object


Iteration 1
Global moving avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,65.666667,61.000000,61.000000,61.000000,61.000000,75.000000,75.000000,75.000000
1,delayFactor,1.237504,1.106782,1.079755,1.079755,1.079755,1.525975,1.525975,1.525975


...

,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,68.987013,61.000000,61.000000,61.000000,61.000000,68.000000,123.000000,126.0
1,delayFactor,1.315332,1.235294,1.040404,1.050462,1.143574,1.340909,1.779615,2.8


Waiting time description from legs: count     77.000000
mean      68.987013
std       16.618390
min       61.000000
5%        61.000000
25%       61.000000
50%       61.000000
75%       61.000000
95%      103.800000
max      126.000000
Name: waitTime, dtype: float64
Delay factor average from legs: count    77.000000
mean      1.315332
std       0.306539
min       1.040404
5%        1.058313
25%       1.149563
50%       1.235294
75%       1.340909
95%       1.750297
max       2.800000
Name: delayFactor, dtype: float64


Waiting time description from legs: count     77.000000
mean      68.987013
std       16.618390
min       61.000000
5%        61.000000
25%       61.000000
50%       61.000000
75%       61.000000
95%      103.800000
max      126.000000
Name: waitTime, dtype: float64
Delay factor average from legs: count    77.000000
mean      1.315332
std       0.306539
min       1.040404
5%        1.058313
25%       1.149563
50%       1.235294
75%       1.340909
95%       1.750297
max  

,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,71.522124,61.000000,61.000000,61.000000,61.00000,75.000000,126.000000,144.0
1,delayFactor,1.299465,1.231336,1.001562,1.043992,1.11157,1.340909,1.750297,2.8


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,71.522124,61.000000,61.000000,61.000000,61.00000,75.000000,126.000000,144.0
1,delayFactor,1.299465,1.231336,1.001562,1.043992,1.11157,1.340909,1.750297,2.8


Waiting time description from legs: count    113.000000
mean      71.522124
std       20.099350
min       61.000000
5%        61.000000
25%       61.000000
50%       61.000000
75%       75.000000
95%      124.200000
max      144.000000
Name: waitTime, dtype: float64
Delay factor average from legs: count    113.000000
mean       1.299465
std        0.285312
min        1.001562
5%         1.047729
25%        1.111570
50%        1.231336
75%        1.340909
95%        1.750297
max        2.800000
Name: delayFactor, dtype: float64


Waiting time description from legs: count    113.000000
mean      71.522124
std       20.099350
min       61.000000
5%        61.000000
25%       61.000000
50%       61.000000
75%       75.000000
95%      124.200000
max      144.000000
Name: waitTime, dtype: float64
Delay factor average from legs: count    113.000000
mean       1.299465
std        0.285312
min        1.001562
5%         1.047729
25%        1.111570
50%        1.231336
75%        1.340909
95%   

Successive avg method is implemented for the avg in python.

In [12]:
def check_global_successive_avg(global_successive_avg, drt_legs, weight, drt_trips_stats):
    iters_wait_time_avg = []
    for it_drt_leg in drt_legs:
        iters_wait_time_avg.append(it_drt_leg.waitTime.mean())
    #print(iters_wait_time_avg)
    iters_wait_time_successive_avg = []
    for it in range(len(iters_wait_time_avg)):
        if it == 0:
            iters_wait_time_successive_avg.append(iters_wait_time_avg[it])
            continue
        previous_avg = iters_wait_time_successive_avg[it-1]
        if np.isnan(previous_avg):
            iters_wait_time_successive_avg.append(iters_wait_time_avg[it])
            continue
        if np.isnan(iters_wait_time_avg[it]):
            iters_wait_time_successive_avg.append(previous_avg)
            continue
        iters_wait_time_successive_avg.append((1-weight) * previous_avg + weight * iters_wait_time_avg[it])
    
    iters_delay_factor_avg = []
    for it_drt_trips_stats in drt_trips_stats:
        iters_delay_factor_avg.append(it_drt_trips_stats.delayFactor.mean())
    #print(iters_delay_factor_avg)
    iters_delay_factor_successive_avg = []
    for it in range(len(iters_delay_factor_avg)):
        if it == 0:
            iters_delay_factor_successive_avg.append(iters_delay_factor_avg[it])
            continue
        previous_avg = iters_delay_factor_successive_avg[it-1]
        if np.isnan(previous_avg):
            iters_delay_factor_successive_avg.append(iters_delay_factor_avg[it])
            continue
        if np.isnan(iters_delay_factor_avg[it]):
            iters_delay_factor_successive_avg.append(previous_avg)
            continue
        iters_delay_factor_successive_avg.append((1-weight) * previous_avg + weight * iters_delay_factor_avg[it])
    
            
    for it, it_global_successive_avg in enumerate(global_successive_avg):
        print('Iteration ' + str(it))
        print('Global successive avg: ')
        display(it_global_successive_avg)
        print('Wait time average from legs: ' + str(iters_wait_time_successive_avg[it]))
        print('Delay factor average from legs: ' + str(iters_delay_factor_successive_avg[it]))
        print('\n')

In [13]:
check_global_successive_avg(no_rej_4000['global_successive_avg'], no_rej_4000['drt_legs'], 0.5, no_rej_4000['drt_trips_stats'])

Iteration 0
Global successive avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,delayFactor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Iteration 0
Global successive avg: 
Wait time average from legs: nan
Delay factor average from legs: nan


Iteration 1
Global successive avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,delayFactor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Wait time average from legs: nan
Delay factor average from legs: nan


Iteration 1
Global successive avg: 


...

Wait time average from legs: 73.0234485479798
Delay factor average from legs: 1.2875064332050643




,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,69.102453,61.000000,61.000000,61.000000,61.000000,72.750000,113.475000,114.750000
1,delayFactor,1.309487,1.237592,1.050242,1.054449,1.135006,1.367142,2.166024,2.509534


Wait time average from legs: 69.10245265151516
Delay factor average from legs: 1.3094873817939887


Iteration 5
Global successive avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,73.023449,61.000000,61.000000,61.000000,61.000000,81.25000,125.762500,129.375000
1,delayFactor,1.287506,1.220671,1.025902,1.044513,1.103864,1.35236,1.980149,2.276506


Wait time average from legs: 73.0234485479798
Delay factor average from legs: 1.2875064332050643


